# Partie recommandation
Réalisée par MUNOZ Matéo.

In [1]:
import pandas as pd
import sklearn as sk

In [2]:
data_product = pd.read_csv('../datasets/olist_products_dataset.csv')
data_order_item = pd.read_csv('../datasets/olist_order_items_dataset.csv')
data_customer = pd.read_csv('../datasets/olist_customers_dataset.csv')
data_review = pd.read_csv('../datasets/olist_order_reviews_dataset.csv')
data_order = pd.read_csv('../datasets/olist_orders_dataset.csv')
data_order_payments = pd.read_csv("../datasets/olist_order_payments_dataset.csv")


### Jeu de données order

Pour ce jeu, on va drop les colonnes qui ne nous serviront pas dans notre analyse. Notamment, on drop les colonnes qui traitent du statut de livraison.

In [3]:
print(len(data_order))
data_order.drop_duplicates(inplace=True)
print(len(data_order))

99441
99441


In [4]:
print(data_order.isnull().sum())

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


In [5]:
data_order.drop(["order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"], axis=1, inplace=True)
data_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39


On va aussi transformer les timestamp en datetime, car ce sont actuellement des chaînes de charactères

In [6]:
data_order["order_purchase_timestamp"] = pd.to_datetime(data_order["order_purchase_timestamp"])
data_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  99441 non-null  object        
 1   customer_id               99441 non-null  object        
 2   order_status              99441 non-null  object        
 3   order_purchase_timestamp  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ MB


### Jeu de données customer

In [7]:
print(len(data_customer))
data_customer.drop_duplicates(inplace=True)
print(len(data_customer))

data_customer.isnull().sum()

99441
99441


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

### Jeu de données order_item

Pour ce jeu de données, on va aussi drop la colonne qui concerne les livraisons.

In [8]:
print(len(data_order_item))
data_order_item.drop_duplicates(inplace=True)
print(len(data_order_item))

print()

print(data_order_item.isnull().sum())

print()

data_order_item.drop("shipping_limit_date", axis=1, inplace=True)
data_order_item.head()

112650
112650

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64



,order_id,order_item_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


### Jeu de données order_payments

In [9]:
print(len(data_order_payments))
data_order_payments.drop_duplicates(inplace=True)
print(len(data_order_payments))

print()

print(data_order_payments.isnull().sum())

103886
103886

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64


### Jeu de données product

In [10]:
print(len(data_product))
data_product.drop_duplicates(inplace=True)
print(len(data_product))

data_product.isnull().sum()

32951
32951


product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [11]:
data_product = data_product[["product_id", "product_category_name"]]
data_product.head(5)

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [12]:
translate_df = pd.read_csv("../datasets/product_category_name_translation.csv")

translate_dict = dict()
for record in translate_df.to_records():
    translate_dict[record[1]] = record[2]

translate_dict

data_product["product_category_name"] = data_product["product_category_name"].replace(translate_dict)
data_product.head(10)

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares
5,41d3672d4792049fa1779bb35283ed13,musical_instruments
6,732bd381ad09e530fe0a5f457d81becb,cool_stuff
7,2548af3e6e77a690cf3eb6368e9ab61e,furniture_decor
8,37cc742be07708b53a98702e77a21a02,home_appliances
9,8c92109888e8cdf9d66dc7e463025574,toys


### Jeu de données order_review

In [13]:
print(len(data_review))
data_review.drop_duplicates(inplace=True)
print(len(data_review))

99224
99224


In [14]:
data_review.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


Beaucoup de review n'ont pas de commentaire et de titre, on va donc drop les colonnes `review_comment_title` et `review_comment_message`.
On va aussi drop les colonnes de timestamp car elles ne sont pas utiles à notre système de recommandations.

In [15]:
data_review.drop("review_comment_title", axis=1, inplace=True)
data_review.drop("review_comment_message", axis=1, inplace=True)
data_review.drop("review_creation_date", axis=1, inplace=True)
data_review.drop("review_answer_timestamp", axis=1, inplace=True)
data_review.head()

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


### Fusion des données obtenues

In [16]:
final_data = data_order.merge(data_order_item, on="order_id", how="left")
final_data = final_data.merge(data_order_payments, on="order_id", how="outer")
final_data = final_data.merge(data_product, on="product_id", how="outer")
final_data = final_data.merge(data_customer, on="customer_id", how="outer")
final_data = final_data.merge(data_review, on="order_id", how="outer")

In [17]:
final_data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_item_id,product_id,seller_id,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,1.0,credit_card,2.0,72.19,cool_stuff,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,97ca439bc427b48bc1cd7177abe71365,5.0
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,1.0,credit_card,3.0,259.83,pet_shop,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,7b07bacd811c4117b742569b04ce3580,4.0
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,1.0,credit_card,5.0,216.87,furniture_decor,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,0c5b33dea94867d1ac402749e5438e8b,5.0
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,1.0,credit_card,2.0,25.78,perfumery,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,f4028d019cb58564807486a6aaf33817,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,1.0,credit_card,3.0,218.04,garden_tools,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,940144190dcba6351888cafa43f3a3a5,5.0


## Explication des données obtenues

On va effectuer une série d'analyse pour expliquer les données que l'on a obtenue.

### Review score

In [18]:
import plotly.express as px

px.bar(final_data['review_score'].value_counts())

On observe que la grande majorité des notes sont de 5, et surtout que peut de notes se situent dans les valeurs intermédiaires (2 et 3). **TODO décider de comment faire le systeme de reco (normalisation des score ou binarisation ??)**.

### Payment

In [19]:
seuils = [10, 100, 500, 1000]

payment_data = {
    f"> {seuil}": final_data[final_data['payment_value'] > seuil].shape[0] 
    for seuil in seuils
}

payment_df = pd.DataFrame(list(payment_data.items()), columns=['payment_category', 'count'])

fig = px.bar(payment_df, x='payment_category', y='count', title="Répartition des paiements supérieurs à différents seuils")

fig.show()

### Autre détail sur les données

In [20]:
print(f'Il y a {len(pd.unique(final_data["product_id"]))} produit unique.')
print(f'Il y a {len(pd.unique(final_data["customer_id"]))} client unique.')
print(f'Il y a {len(pd.unique(final_data["seller_id"]))} vendeur unique.')

Il y a 32952 produit unique.
Il y a 99441 client unique.
Il y a 3096 vendeur unique.


### Système de recommendation
On va commencer par retirer les données qui ne nous servent pas pour faire la recommendation, puis on va transformer celle dont on a besoin en nombre (comme les identifiants par exemple qui sont des strings).

In [21]:
import json
from sklearn.preprocessing import LabelEncoder

customer_encoder = LabelEncoder()
product_encoder = LabelEncoder()

data_ratings = final_data[['customer_unique_id', 'product_id', 'review_score']].copy()
data_ratings.dropna(inplace=True)
data_ratings['customer_unique_id'] = customer_encoder.fit_transform(data_ratings['customer_unique_id'])

data_ratings['product_id'] = product_encoder.fit_transform(data_ratings['product_id'])
product_id_mapping = dict(zip(product_encoder.classes_, product_encoder.transform(product_encoder.classes_)))

data_ratings.head()

,customer_unique_id,product_id,review_score
0,50046,8577,5.0
1,87105,29455,4.0
2,20860,25545,5.0
3,65099,15243,4.0
4,37376,21967,5.0


On va normaliser les scores de review.

In [22]:
from sklearn.preprocessing import MinMaxScaler

score_normalizer = MinMaxScaler()

data_ratings['review_score'] = score_normalizer.fit_transform(data_ratings[['review_score']])

data_ratings.head()

,customer_unique_id,product_id,review_score
0,50046,8577,1.00
1,87105,29455,0.75
2,20860,25545,1.00
3,65099,15243,0.75
4,37376,21967,1.00


In [23]:
from surprise import SVD, Dataset, Reader
import joblib


reader = Reader(rating_scale=(data_ratings['review_score'].min(), data_ratings['review_score'].max()))

data_ratings_surprise = Dataset.load_from_df(data_ratings, reader)

trainset = data_ratings_surprise.build_full_trainset()

model = SVD(n_factors=15)
model.fit(trainset)

feature_mat = model.qi

product = [trainset.to_raw_iid(x) for x in range(len(feature_mat))]

feature_df = pd.DataFrame(feature_mat, index=product)

joblib.dump(model, 'svd_model.pkl')

feature_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
8577,0.183565,0.024145,0.045547,-0.004724,0.016029,-0.008730,-0.009253,0.043641,-0.054960,-0.224110,-0.021603,0.059242,-0.063924,0.062288,-0.041629
29455,0.018835,0.020430,0.275007,-0.195758,-0.121168,-0.102108,0.097023,0.122745,0.009657,0.047812,-0.102446,0.142964,0.043789,-0.036320,0.104983
25545,-0.016154,-0.034672,-0.255046,0.194215,0.086992,0.008047,0.012812,-0.060349,0.111067,0.163296,0.062341,0.043886,-0.048662,-0.015919,-0.085441
15243,0.158780,0.161929,-0.079784,-0.037947,-0.069033,0.230324,0.003739,-0.008333,-0.124567,-0.021313,-0.112758,0.121175,-0.167998,-0.099174,-0.023057
21967,-0.025767,-0.053194,-0.200928,-0.097513,0.017803,-0.054752,0.288074,-0.038844,0.082566,0.037757,0.007457,0.204120,0.119157,0.052432,-0.287248


In [24]:
from sklearn.manifold import TSNE

tsne = TSNE(2, perplexity=10)

reduced_feat = tsne.fit_transform(feature_df)

tsne_df = pd.DataFrame(reduced_feat, index=product, columns=['Factor 1', 'Factor 2'])


px.scatter(tsne_df, 
    x='Factor 1', 
    y='Factor 2',
    hover_name=tsne_df.index)

In [ ]:
from surprise import Reader
from surprise import Dataset


def recommend_product_for_customer(model, customer_id, data_ratings, n_recommendations=5):
    reader = Reader()
    data_ratings_surprise = Dataset.load_from_df(data_ratings, reader)
    
    trainset = data_ratings_surprise.build_full_trainset()

    model.fit(trainset)

    rated_product_row = data_ratings[data_ratings['customer_unique_id'] == customer_id]
    
    all_product = data_ratings['product_id'].unique()
    rated_product = rated_product_row['product_id'].unique()
    product_to_predict = [product for product in all_product if product not in rated_product]
    
    predictions = [model.predict(customer_id, movie) for movie in product_to_predict]
    
    recommended_products = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_recommendations]
    
    rated_prod_final = [(prod, score) for prod, score in rated_product_row[['product_id', 'review_score']].values.tolist()]

    results = {
        'Rated Product': rated_prod_final,
        'Recommended Product': [(int(pred.iid), pred.est) for pred in recommended_products]
    }

    return results

random_customer_id = data_ratings['customer_unique_id'].sample(1).iloc[0]

print(f"Client ID aléatoire : {random_customer_id}")
loaded_model = joblib.load('svd_model.pkl')
results = recommend_product_for_customer(loaded_model, random_customer_id, data_ratings)

print("Rated Product:")
for product, rating in results['Rated Product']:
    product = int(product)
    for k, v in product_id_mapping.items():
        if v == int(product):
            product_info = final_data[final_data['product_id'] == k]
            break
        
    print(f"{product} - {product_info['product_category_name'].values[0]}: {rating}")

print("\nRecommended Product:")
for product, est_rating in results['Recommended Product']:
    for k, v in product_id_mapping.items():
        if v == int(product):
            product_info = final_data[final_data['product_id'] == k]
            break
    print(f"{product} - {product_info['product_category_name'].values[0]}: Estimated Rating = {est_rating:.2f}")

Client ID aléatoire : 75569
       customer_unique_id  product_id  review_score
15077               75569       21196          0.25
15078               75569       21196          0.25
15079               75569       21196          0.25
[21196]
Rated Product:
21196 - computers_accessories: 0.25
21196 - computers_accessories: 0.25
21196 - computers_accessories: 0.25

Recommended Product:
8577 - cool_stuff: Estimated Rating = 1.00
29455 - pet_shop: Estimated Rating = 1.00
25545 - furniture_decor: Estimated Rating = 1.00
15243 - perfumery: Estimated Rating = 1.00
21967 - garden_tools: Estimated Rating = 1.00
